In [1]:
from __future__ import division
import os, scipy.io, time, pickle, random
import tensorflow as tf
# import tensorflow.contrib.slim as slim
import numpy as np
import rawpy
# import glob
import tensorlayer as tl
from tensorlayer.prepro import imresize


from dark_utils import *

# import os, time, pickle, random, time
from datetime import datetime
from time import localtime, strftime
import logging, scipy
from model2 import SRGAN_g, SRGAN_d, Vgg19_simple_api
from utils2 import *
from config2 import config, log_config

from PIL import Image


In [2]:
RATIO = 20
INPUT_IMG = 'test_data/sony_a7s_ii_38.arw'

## Load the first checkpoint

In [3]:
sess1 = tf.Session()
in_image = tf.placeholder(tf.float32, [None, None, None, 4])
out_image = network(in_image)
saver = tf.train.Saver()
sess1.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state('checkpoint1/Sony')
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess1, ckpt.model_checkpoint_path)

loaded checkpoint1/Sony/model.ckpt
INFO:tensorflow:Restoring parameters from checkpoint1/Sony/model.ckpt


## Load the second checkpoint

In [4]:
t_image = tf.placeholder('float32', [1, None, None, 3], name='input_image')
net_g = SRGAN_g(t_image, is_train=False, reuse=False)

sess2 = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess2)
tl.files.load_and_assign_npz(sess=sess2, name='checkpoint2' + '/g_srgan.npz', network=net_g)

[TL] InputLayer  SRGAN_g/in: (1, ?, ?, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (1, ?, ?, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b

[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/add3: size: (1, ?, ?, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out: n_filter: 3 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh
[TL] WARNING: Function: `tensorlayer.layers.utils.initialize_global_variables` (in file: /home/hwa125/.local/lib/python3.6/site-packages/tensorlayer/layers/utils.py) is deprecated and will be removed after 2018-09-30.
Instructions for upd

## Run the first model

In [5]:
raw = rawpy.imread(INPUT_IMG)
input_full = np.expand_dims(pack_raw(raw), axis=0) * RATIO
input_full = np.minimum(input_full, 1.0)
output = sess1.run(out_image, feed_dict={in_image: input_full})
output = np.minimum(np.maximum(output, 0), 1)

output = output[0, :, :, :]
output = output * 255

In [6]:
sess1.close()

## Run the second model

(first resize the image to 1/8 of the original)

In [7]:
output = imresize(output, (int(output.shape[0]/8), int(output.shape[1]/8)))
output = (output / 127.5) - 1
output2 = sess2.run(net_g.outputs, {t_image: [output]})

## save the output

In [8]:
tl.vis.save_image(output2[0], 'output' + '/valid_gen.png')

In [9]:
output2.shape

(1, 1424, 2128, 3)